In [1]:
import numpy as np
import numpy as np
from numpy.random import randn

rate, div, vol, spot, strike, barrier = 0.05, 0.03, 0.15, 100.0, 120.0, 150

def create_model():
    return np.array([rate, div, vol, spot, strike, barrier])

n_paths = 2 ** 20
T = 3.0

In [2]:
default_model = create_model()

## 0. DAL
------------------

In [3]:
# !pip install dal-python -U

from dal import *
import dal
dal.__version__

'0.3.5'

In [4]:
today = Date_(2022, 9, 15)
EvaluationDate_Set(today)

maturity = today.AddDays(int(365 * T))
maturity
freq = "1M"

In [5]:
%%time
event_dates = ["STRIKE", "BARRIER", today]
events = [f"{strike:.2f}", f"{barrier:.2f}", "alive = 1"]
event_dates.append(f"START: {today} END: {maturity} FREQ: {freq}")
events.append("if spot() >= BARRIER:0.1 then alive = 0 end")
event_dates.append(maturity)
events.append(f"if spot() >= BARRIER:0.1 then alive = 0 end\ncall pays alive * MAX(spot() - STRIKE, 0.0)")

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
%%time

product = Product_New(event_dates, events)
model = BSModelData_New(spot, vol, rate, div)

# only price
res = MonteCarlo_Value(product, model, n_paths, "sobol", False, False)
dict(res)

CPU times: total: 3.05 s
Wall time: 425 ms


{'value': 1.5057628229020656}

In [7]:
%%time

# price with derivatives
res = MonteCarlo_Value(product, model, n_paths, "sobol", False, True)
dict(res)

CPU times: total: 8.44 s
Wall time: 1.08 s


{'d_div': -19.381229696569612,
 'd_rate': 14.865501003015163,
 'd_spot': 0.04974902318831637,
 'd_vol': -7.227015324773113,
 'value': 1.5052428978514838}

## 1. DAL.JAX
----------------

In [8]:
import jax
import jax.numpy as jnp
import math

In [9]:
if freq == "1M":
    dt, NT = 1.0 / 12, 3 * 12
elif freq == "1W":
    dt, NT = 1.0 / 51, 3 * 51

In [10]:
def compute_barrier_price_jax(model, delta_t=dt, NT = NT, M=n_paths, key=jax.random.PRNGKey(1)):
    """
    Estimate the price of the up barrieroption using Monte Carlo.
    """
    # Set up
    μ, d, σ, S, K, b = model
    log_b = jnp.log(b)
    s = jnp.full(M, jnp.log(S))
    alive = jnp.full(M, 1.0)
    not_alive = jnp.full(M, 0.0)
    subkey = key
    for _ in range(NT):
        key, subkey = jax.random.split(subkey)
        Z = jax.random.normal(key, (1, M))
        s = s + (μ - d - 0.5 * σ * σ) * delta_t +  σ * jnp.sqrt(delta_t) * Z[0, :]
        alive = jnp.where(s >= log_b, not_alive, alive)
    expectation = jnp.mean(alive * jnp.maximum(jnp.exp(s) - K, 0))
    return jnp.exp(-μ * delta_t * NT) * expectation

compute_barrier_price_jax_jit = jax.jit(compute_barrier_price_jax)
_ = compute_barrier_price_jax(default_model).block_until_ready()
_ = compute_barrier_price_jax_jit(default_model).block_until_ready()

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [11]:
%%time

# only price without jit
compute_barrier_price_jax(default_model).block_until_ready()

CPU times: total: 1.2 s
Wall time: 518 ms


Array(1.5024601, dtype=float32)

In [12]:
compute_barrier_price_jax(default_model).block_until_ready()

Array(1.5024601, dtype=float32)

In [13]:
%%time

# only price without jit
compute_barrier_price_jax_jit(default_model).block_until_ready()

CPU times: total: 500 ms
Wall time: 173 ms


Array(1.5024601, dtype=float32)

In [14]:
compute_barrier_value_and_grad = jax.value_and_grad(compute_barrier_price_jax)
compute_barrier_value_and_grad_jit = jax.value_and_grad(jax.jit(compute_barrier_price_jax))

_ = compute_barrier_value_and_grad(default_model)[0].block_until_ready()
_ = compute_barrier_value_and_grad_jit(default_model)[0].block_until_ready()

In [15]:
%%time

# price and derivatives without jit
# TODO: the output numbers of greeks are not correct
result = compute_barrier_value_and_grad(default_model)
result[0].block_until_ready()
result[1].block_until_ready()
result

CPU times: total: 1.55 s
Wall time: 927 ms


(Array(1.5024601, dtype=float32),
 Array([ 50.393784  , -54.901165  ,  21.463438  ,   0.18300392,
         -0.13998276,   0.        ], dtype=float32))

In [16]:
%%time

# price and derivatives with jit
# TODO: the output numbers of greeks are not correct
result = compute_barrier_value_and_grad_jit(default_model)
result[0].block_until_ready()
result[1].block_until_ready()
result

CPU times: total: 1.38 s
Wall time: 341 ms


(Array(1.5024601, dtype=float32),
 Array([ 50.393784  , -54.901165  ,  21.463436  ,   0.18300392,
         -0.13998276,   0.        ], dtype=float32))